<a href="https://colab.research.google.com/github/a-forty-two/diamler_17_jan/blob/main/03_nlp_piidetection_azure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install azure-ai-textanalytics==5.1.0
!pip install azure-storage-blob

     |████████████████████████████████| 356 kB 24.5 MB/s 
     |████████████████████████████████| 3.6 MB 58.6 MB/s 


In [2]:
key = "dbe17db2747c4135815c88519deb9dfa"
endpoint = "https://iyakajhoka.cognitiveservices.azure.com/"

from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

ta_credential = AzureKeyCredential(key)
client = TextAnalyticsClient(
            endpoint=endpoint, 
            credential=ta_credential)


In [4]:
from azure.storage.blob import BlobServiceClient
azaccesskey = 'OfNDTm/gkMeRZ0L6mgKedUChEKkg8Wc0fKWplJS3EZK4EwS+1zk4GsaxCfiF4X7C4Gxdeo+Rd/41jH+nbG3LfQ=='
service = BlobServiceClient(account_url="https://shaktimanlikesbananana.blob.core.windows.net", credential=azaccesskey)

In [7]:
from azure.storage.blob import BlobClient
constring = 'DefaultEndpointsProtocol=https;AccountName=shaktimanlikesbananana;AccountKey=OfNDTm/gkMeRZ0L6mgKedUChEKkg8Wc0fKWplJS3EZK4EwS+1zk4GsaxCfiF4X7C4Gxdeo+Rd/41jH+nbG3LfQ==;EndpointSuffix=core.windows.net'
blob = BlobClient.from_connection_string(conn_str=constring, container_name="text", blob_name="pii.txt")
# blob-> written in binary
with open("pii.txt", "wb") as my_blob:
    blob_data = blob.download_blob()
    blob_data.readinto(my_blob)

In [8]:
# text-> read as text
mydata = open("pii.txt", "rt").read()
mydata

'My name is Pikachu Pandey. My phone number is 991-111-1111.\n\nI live at 221 Baker Street, Ghost Lake, Screamville. \n'

In [16]:
mydata = mydata.replace("\n","")
sentokens = mydata.split('.')
print(sentokens)

['My name is Pikachu Pandey', ' My phone number is 991-111-1111', 'I live at 221 Baker Street, Ghost Lake, Screamville', ' ']


In [17]:


# Example method for detecting sensitive information (PII) from text 
def pii_recognition_example(client, mydata):
    documents = mydata
    response = client.recognize_pii_entities(documents, language="en")
    result = [doc for doc in response if not doc.is_error]
    for doc in result:
        print("Redacted Text: {}".format(doc.redacted_text))
        for entity in doc.entities:
            print("Entity: {}".format(entity.text))
            print("\tCategory: {}".format(entity.category))
            print("\tConfidence Score: {}".format(entity.confidence_score))
            print("\tOffset: {}".format(entity.offset))
            print("\tLength: {}".format(entity.length))


In [18]:
pii_recognition_example(client, sentokens)

Redacted Text: My name is **************
Entity: Pikachu Pandey
	Category: Person
	Confidence Score: 0.94
	Offset: 11
	Length: 14
Redacted Text:  My phone number is ************
Entity: 991-111-1111
	Category: PhoneNumber
	Confidence Score: 0.8
	Offset: 20
	Length: 12
Redacted Text: I live at *****************************************
Entity: 221 Baker Street, Ghost Lake, Screamville
	Category: Address
	Confidence Score: 0.84
	Offset: 10
	Length: 41
